In [1]:
# create project directory (could be any name)
!mkdir genre_test
# create and activate virtual environment
%cd genre_test
!python -m venv .venv_genre
!source .venv_genre/bin/activate
# upgrade pip
!pip install --upgrade pip
# install correct version of of fairseq
!git clone --branch fixing_prefix_allowed_tokens_fn https://github.com/nicola-decao/fairseq
%cd fairseq
!pip install --editable ./
%cd ..
# install transformers
!pip install transformers
# install KITL
!git clone https://github.com/facebookresearch/KILT.git
%cd KILT
!pip install -r requirements.txt
!python setup.py install
%cd ..
# install GENRE
!git clone https://github.com/facebookresearch/GENRE.git
%cd GENRE
!pip install -r requirements.txt
!python setup.py install
%cd ..

mkdir: cannot create directory ‘genre_test’: File exists
/content/genre_test
Error: Command '['/content/genre_test/.venv_genre/bin/python3', '-Im', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: .venv_genre/bin/activate: No such file or directory
fatal: destination path 'fairseq' already exists and is not an empty directory.
/content/genre_test/fairseq
Obtaining file:///content/genre_test/fairseq
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
/content/genre_test
fatal: destination path 'KILT' already exists and is not an empty directory.
/content/genre_test/KILT
running install
running bdist_egg
running egg_info
writing kilt.egg-info/PKG-INFO
writing dependency_links to kilt.egg-info/dependency_links.txt
writing top-level names to kilt.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'kilt.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
r

In [2]:
%pwd

'/content/genre_test'

In [3]:
# %cd ..

In [4]:
import pickle
from genre.trie import Trie
from genre.hf_model import GENRE

In [5]:
!wget http://dl.fbaipublicfiles.com/GENRE/kilt_titles_trie_dict.pkl

--2021-06-24 02:50:45--  http://dl.fbaipublicfiles.com/GENRE/kilt_titles_trie_dict.pkl
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215214973 (205M) [application/octet-stream]
Saving to: ‘kilt_titles_trie_dict.pkl.1’

kilt_titles_trie_di 100%[===================>] 205.24M  12.2MB/s    in 18s     

2021-06-24 02:51:04 (11.2 MB/s) - ‘kilt_titles_trie_dict.pkl.1’ saved [215214973/215214973]



In [6]:
with open("kilt_titles_trie_dict.pkl", "rb") as f:
    trie = Trie.load_from_dict(pickle.load(f))

1. Document Retieval (Wikipedia)

In [7]:
# !wget http://dl.fbaipublicfiles.com/GENRE/hf_wikipage_retrieval.tar.gz

In [8]:
# !tar -zxvf hf_wikipage_retrieval.tar.gz

In [9]:
# DR_model = GENRE.from_pretrained("hf_wikipage_retrieval").eval()

In [10]:
# sentences = ["In 1921, Einstein received a Nobel Prize."]

# DR_model.sample(
#     sentences,
#     prefix_allowed_tokens_fn=lambda batch_id, sent: trie.get(sent.tolist()),
# )

2. Entity Disambiguation

In [11]:
# !wget http://dl.fbaipublicfiles.com/GENRE/hf_entity_disambiguation_blink.tar.gz
# !tar -zxvf hf_entity_disambiguation_blink.tar.gz

In [12]:
# ED_model_1 = GENRE.from_pretrained("hf_entity_disambiguation_blink").eval()

In [13]:
# sentences = ["Cristiano was an outstanding [START_ENT] soccer player [END_ENT]."]

# ED_model_1.sample(
#     sentences,
#     prefix_allowed_tokens_fn=lambda batch_id, sent: trie.get(sent.tolist()),
# )

In [14]:
# !wget http://dl.fbaipublicfiles.com/GENRE/hf_entity_disambiguation_aidayago.tar.gz
# !tar -zxvf hf_entity_disambiguation_aidayago.tar.gz

In [15]:
# ED_model_2 = GENRE.from_pretrained("hf_entity_disambiguation_aidayago").eval()

In [16]:
# sentences = ["Cristiano was an outstanding [START_ENT] soccer player [END_ENT]."]

# ED_model_2.sample(
#     sentences,
#     prefix_allowed_tokens_fn=lambda batch_id, sent: trie.get(sent.tolist()),
# )
# # The result is slightly different from ED_model_1

3. End-to-End Entity Linking

In [17]:
!wget http://dl.fbaipublicfiles.com/GENRE/hf_e2e_entity_linking_wiki_abs.tar.gz
!tar -zxvf hf_e2e_entity_linking_wiki_abs.tar.gz

--2021-06-24 02:51:43--  http://dl.fbaipublicfiles.com/GENRE/hf_e2e_entity_linking_wiki_abs.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 958456496 (914M) [application/gzip]
Saving to: ‘hf_e2e_entity_linking_wiki_abs.tar.gz.1’

hf_e2e_entity_linki 100%[===================>] 914.05M  11.9MB/s    in 78s     

2021-06-24 02:53:02 (11.7 MB/s) - ‘hf_e2e_entity_linking_wiki_abs.tar.gz.1’ saved [958456496/958456496]

hf_e2e_entity_linking_wiki_abs/
hf_e2e_entity_linking_wiki_abs/config.json
hf_e2e_entity_linking_wiki_abs/pytorch_model.bin
hf_e2e_entity_linking_wiki_abs/vocab.json
hf_e2e_entity_linking_wiki_abs/merges.txt


In [18]:
E2EEL_model_1 = GENRE.from_pretrained("hf_e2e_entity_linking_wiki_abs").eval()

In [19]:
from genre.entity_linking import get_end_to_end_prefix_allowed_tokens_fn_hf as get_prefix_allowed_tokens_fn

sentences = ["In 1921, Einstein received a Nobel Prize in America."]

prefix_allowed_tokens_fn = get_prefix_allowed_tokens_fn(E2EEL_model_1, sentences)

E2EEL_model_1.sample(
    sentences,
    prefix_allowed_tokens_fn=prefix_allowed_tokens_fn,
)

[[{'logprob': tensor(-0.8458),
   'text': 'In { 1921 } [ List of Nobel laureates in Physics by year of publication ], { Einstein } [ Albert Einstein ] received a { Nobel Prize } [ Nobel Prize in Physiology or Medicine ] in { America } [ United States ].'}],
 [{'logprob': tensor(-1.0061),
   'text': 'In { 1921 } [ List of Nobel laureates in Physics by year of publication ], { Einstein } [ Albert Einstein ] received a { Nobel Prize } [ Nobel Prize in Physics ] in { America } [ United States in the Nobel Prize laureates of 1921 and 1922 ].'}],
 [{'logprob': tensor(-1.4678),
   'text': 'In { 1921 } [ List of Nobel laureates in Physics by year of publication ], { Einstein } [ Albert Einstein ] received a { Nobel Prize } [ Nobel Prize in Physics ] in { America } [ United States in the Nobel Prize laureates of 1921–1924, P. G. W. Fields Award, Prize No. 1, Mathematical Mathematical Trio and Nobel Memorial Prize in Physiology or Physiology (Euclidean-Aristotle) ].'}],
 [{'logprob': tensor(-1.4

In [20]:
!wget http://dl.fbaipublicfiles.com/GENRE/hf_e2e_entity_linking_aidayago.tar.gz
!tar -zxvf hf_e2e_entity_linking_aidayago.tar.gz

--2021-06-24 02:56:02--  http://dl.fbaipublicfiles.com/GENRE/hf_e2e_entity_linking_aidayago.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 958460503 (914M) [application/gzip]
Saving to: ‘hf_e2e_entity_linking_aidayago.tar.gz.1’

hf_e2e_entity_linki 100%[===================>] 914.06M  11.9MB/s    in 78s     

2021-06-24 02:57:20 (11.7 MB/s) - ‘hf_e2e_entity_linking_aidayago.tar.gz.1’ saved [958460503/958460503]

hf_e2e_entity_linking_aidayago/
hf_e2e_entity_linking_aidayago/config.json
hf_e2e_entity_linking_aidayago/pytorch_model.bin
hf_e2e_entity_linking_aidayago/vocab.json
hf_e2e_entity_linking_aidayago/merges.txt


In [21]:
E2EEL_model_2 = GENRE.from_pretrained("hf_e2e_entity_linking_aidayago").eval()

In [22]:
sentences = ["In 1921, Einstein received a Nobel Prize in America."]

prefix_allowed_tokens_fn = get_prefix_allowed_tokens_fn(E2EEL_model_2, sentences)

E2EEL_model_2.sample(
    sentences,
    prefix_allowed_tokens_fn=prefix_allowed_tokens_fn,
)
# The result is slightly different from E2EEL_model_1

[[{'logprob': tensor(-0.6685),
   'text': 'In 1921, { Einstein } [ Albert Einstein ] received a { Nobel Prize } [ Nobel Prize ] in { America } [ United States ].'}],
 [{'logprob': tensor(-0.7592),
   'text': 'In 1921, { Einstein } [ Albert Einstein ] received a { Nobel Prize } [ Nobel Prize in Physics ] in { America } [ United States ].'}],
 [{'logprob': tensor(-0.7706),
   'text': 'In 1921, { Einstein } [ Albert Einstein ] received a { Nobel Prize } [ Nobel Prize in Physiology or Medicine ] in { America } [ United States ].'}],
 [{'logprob': tensor(-1.0445),
   'text': 'In 1921, { Einstein } [ Albert Einstein ] received a { Nobel Prize } [ Nobel Prize in Physiology or Medicine ] in { America } [ United States in the 1921 Nobel Prize ].'}],
 [{'logprob': tensor(-1.1765),
   'text': 'In 1921, { Einstein } [ Albert Einstein ] received a { Nobel Prize } [ Nobel Prize in Physiology or Medicine ] in { America } [ United States in the 1921 Nobel Prize Winners ].'}]]

In [23]:
# constrain the mentions with a prefix tree (no constrains on candidates)
prefix_allowed_tokens_fn = get_prefix_allowed_tokens_fn(
    E2EEL_model_2,
    sentences,
    mention_trie=Trie([
        E2EEL_model_2.encode(e)[1:].tolist()
        for e in [" Nobel Prize"]
    ])
)

E2EEL_model_2.sample(
    sentences,
    prefix_allowed_tokens_fn=prefix_allowed_tokens_fn,
)

[[{'logprob': tensor(-1.8145),
   'text': 'In 1921, Einstein received a { Nobel Prize } [ Nobel Prize in Physiology or Medicine ] in America.'}],
 [{'logprob': tensor(-1.9818),
   'text': 'In 1921, Einstein received a { Nobel Prize } [ Nobel Prize in Physiology or Physiology ] in America.'}],
 [{'logprob': tensor(-2.0606),
   'text': 'In 1921, Einstein received a { Nobel Prize } [ Nobel Prize in Physiology or Chemistry ] in America.'}],
 [{'logprob': tensor(-2.0755),
   'text': 'In 1921, Einstein received a { Nobel Prize } [ Nobel Prize in Physiology or Physics ] in America.'}],
 [{'logprob': tensor(-2.1495),
   'text': 'In 1921, Einstein received a { Nobel Prize } [ Nobel Prize in Physiology or Medicine in 1921 ] in America.'}]]

In [24]:
# constrain the candidates with a prefix tree (no constrains on mentions)
prefix_allowed_tokens_fn = get_prefix_allowed_tokens_fn(
    E2EEL_model_2,
    sentences,
    candidates_trie=Trie([
        E2EEL_model_2.encode(" }} [ {} ]".format(e))[1:].tolist()
        for e in ["Albert Einstein", "Nobel Prize in Physics", "USA", "NIL"]
    ])
)

E2EEL_model_2.sample(
    sentences,
    prefix_allowed_tokens_fn=prefix_allowed_tokens_fn,
)

[[{'logprob': tensor(-1.1565),
   'text': 'In 1921, { Einstein } [ Albert Einstein ] received a { Nobel Prize } [ Nobel Prize in Physics ] in { America } [ USA ].'}],
 [{'logprob': tensor(-1.3672),
   'text': 'In 1921, { Einstein } [ Albert Einstein ] received a { Nobel Prize } [ Nobel Prize in Physics ] in { America. } [ USA ]'}],
 [{'logprob': tensor(-1.4649),
   'text': 'In 1921, { Einstein } [ Albert Einstein ] received a { Nobel } [ Nobel Prize in Physics ] Prize in { America } [ USA ].'}],
 [{'logprob': tensor(-1.9403),
   'text': 'In 1921, { Einstein } [ Albert Einstein ] received a { Nobel Prize } [ Nobel Prize in Physics ] in { America } [ USA ] {. } [ NIL ]'}],
 [{'logprob': tensor(-2.1789),
   'text': 'In 1921, { Einstein } [ Albert Einstein ] received a { Nobel } [ Nobel Prize in Physics ] Prize in { America } [ USA ] {. } [ NIL ]'}]]